**Importando os dados de uma API**

In [1]:
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

In [2]:
import pandas as pd
df = pd.read_json(url)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


**Limpeza e Tratamento de dados**

In [3]:
# Normalizando e removendo colunas irrelevantes
customer_normalizado = pd.json_normalize(df['customer'])
internet_normalizado = pd.json_normalize(df['internet'])
account_normalizado = pd.json_normalize(df['account'])
dados_normalizados = pd.concat([customer_normalizado, internet_normalizado, account_normalizado], axis=1)
dados = pd.concat([df[['Churn']], dados_normalizados], axis=1)
dados.head()


,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,No,Female,0,Yes,Yes,9,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,No,Male,0,No,No,9,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,Yes,Male,0,No,No,4,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,Yes,Male,1,Yes,No,13,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Yes,Female,1,Yes,No,3,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Encoding

In [4]:
# Identificando colunas categóricas
cat_cols = dados.select_dtypes(include='object').columns
cat_cols

Index(['Churn', 'gender', 'Partner', 'Dependents', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'Charges.Total'],
      dtype='object')

In [5]:
# Aplicando One-Hot Encoding
dados_encoding = pd.get_dummies(dados, columns=cat_cols, drop_first=True)
dados_encoding.head()

,SeniorCitizen,tenure,Charges.Monthly,Churn_No,Churn_Yes,gender_Male,Partner_Yes,Dependents_Yes,InternetService_Fiber optic,InternetService_No,...,Charges.Total_995.35,Charges.Total_996.45,Charges.Total_996.85,Charges.Total_996.95,Charges.Total_997.65,Charges.Total_997.75,Charges.Total_998.1,Charges.Total_999.45,Charges.Total_999.8,Charges.Total_999.9
0,0,9,65.6,True,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0,9,59.9,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0,4,73.9,False,True,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,1,13,98.0,False,True,True,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,1,3,83.9,False,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False


Verificação da Proporção de Evasão

In [6]:
# Verificando proporção da variável alvo (Churn)
churn_rate = dados['Churn'].value_counts(normalize=True)
print(churn_rate)

Churn
No     0.711986
Yes    0.257190
       0.030824
Name: proportion, dtype: float64


Normalização ou Padronização

In [7]:
from sklearn.preprocessing import StandardScaler

# Separando variáveis numéricas
num_cols = dados_encoding.select_dtypes(include=['int64', 'float64']).columns

# Aplicando padronização
scaler = StandardScaler()
df_scaled = dados_encoding.copy()
df_scaled[num_cols] = scaler.fit_transform(dados_encoding[num_cols])
df_scaled.head()

,SeniorCitizen,tenure,Charges.Monthly,Churn_No,Churn_Yes,gender_Male,Partner_Yes,Dependents_Yes,InternetService_Fiber optic,InternetService_No,...,Charges.Total_995.35,Charges.Total_996.45,Charges.Total_996.85,Charges.Total_996.95,Charges.Total_997.65,Charges.Total_997.75,Charges.Total_998.1,Charges.Total_999.45,Charges.Total_999.8,Charges.Total_999.9
0,-0.440736,-0.950200,0.029206,True,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,-0.440736,-0.950200,-0.159990,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,-0.440736,-1.153700,0.304702,False,True,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,2.268932,-0.787401,1.104635,False,True,True,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,2.268932,-1.194400,0.636624,False,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False


Análise de correlação

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Matriz de correlação
plt.figure(figsize=(8,5))
sns.heatmap(dados_encoding.corr(), cmap='coolwarm', annot=False, cbar=False)
plt.title("Matriz de Correlação")
plt.tight_layout()
plt.show()

Análise direcionada

In [ ]:
# Boxplot: Tempo de contrato vs Evasão
sns.boxplot(x='Churn', y='tenure', data=dados)
plt.title("Tempo de Contrato × Evasão")
plt.show()

# Scatter plot: Total gasto vs Evasão
sns.scatterplot(x='TotalCharges', y='MonthlyCharges', hue='Churn', data=dados)
plt.title("Total Gasto × Evasão")
plt.show()


Separação de dados

In [ ]:
from sklearn.model_selection import train_test_split

X = dados_encoding.drop('Churn_Yes', axis=1)
y = dados_encoding['Churn_Yes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Criação de modelos

In [ ]:
# Regressão logística com normalização
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)


In [ ]:
# Randon Florest sem normalização
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

Avaliação dos modelos

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def avaliar_modelo(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    print("Acurácia:", accuracy_score(y_test, y_pred))
    print("Precisão:", precision_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("F1-score:", f1_score(y_test, y_pred))
    print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))

avaliar_modelo(model_lr, X_test, y_test)
avaliar_modelo(model_rf, X_test, y_test)


**Interpretação e Conclusão**

O projeto teve como objetivo entender os fatores que influenciam a evasão de clientes da TelecomX. Primeiro, foram removidas colunas irrelevantes como o ID do cliente. Em seguida, as variáveis categóricas foram transformadas em numéricas com one-hot encoding. A proporção de evasão foi de cerca de 26%, indicando um certo desequilíbrio entre as classes.
Foi aplicada normalização para modelos que exigem esse tipo de pré-processamento. A análise de correlação mostrou que variáveis como tempo de contrato, gastos mensais e serviços adicionais têm forte relação com a evasão. Boxplots e gráficos de dispersão confirmaram que clientes com menor tempo de contrato e menor gasto tendem a evadir mais.
Os dados foram divididos em treino e teste (70/30), e dois modelos foram criados: Regressão Logística (com normalização) e Random Forest (sem normalização). O Random Forest teve desempenho superior em todas as métricas: acurácia, precisão, recall e F1-score.
Na interpretação dos modelos, a Regressão Logística mostrou que contratos mensais e ausência de serviços como segurança online aumentam a chance de evasão. O Random Forest indicou que tempo de contrato, valor mensal e serviços adicionais são os fatores mais importantes.
Com base nisso, foram sugeridas estratégias como incentivar contratos mais longos, oferecer serviços adicionais e personalizar ofertas para clientes com baixo engajamento. A combinação dos modelos permitiu uma análise robusta e útil para orientar ações de retenção.
